we will decode the Categories to use catboost in its full limits,because after a search we did we found out that CatBoost might misinterpret the binary-encoded values as having ordinal relationships and it will treat them as numerical features rather than categorical features. which means CatBoost won’t apply its specialized handling for categorical data (e.g., Ordered Target Encoding), which could reduce model performance.

Justification for Choosing SVM

Support Vector Machines (SVM) is a well-established model in text classification due to its ability to handle high-dimensional feature spaces. In this project, book descriptions were transformed into numerical feature vectors using TF-IDF (Term Frequency-Inverse Document Frequency) or word embeddings, making SVM a strong candidate.

Key reasons for selecting SVM:

    1.	Effective in High-Dimensional Spaces: Text data results in a large feature space, and SVM is designed to handle such cases efficiently.
    2.	Strong Generalization Ability: SVM finds the optimal decision boundary between categories, reducing overfitting.
    3.	Works Well with Medium-Sized Datasets: With 3,300 rows, deep learning models would require significantly more data, whereas SVM remains effective with this dataset size.
    4.	Proven Success in Text Categorization: SVM has been widely used in spam detection, sentiment analysis, and document classification
.

Reference for SVM

A foundational study on SVM for text classification is:
Joachims, T. & Universität Dortmund. (1998). Text Categorization with Support Vector Machines: Learning with Many Relevant Features. https://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf

This paper provides theoretical and empirical evidence that SVM is well-suited for categorizing text into distinct classes. It demonstrates that SVM outperforms traditional models such as Naïve Bayes in document classification tasks.




In [14]:
%pip install catboost
%pip install pandas openpyxl xlrd
%pip install torch
%pip install transformers
%pip install numpy
%pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import pickle
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Load dataset
file_path = "/Users/batoolalfouzan/Desktop/books-main/Cleaned Dataset/Book_Cleaned_Dataset_.xls"
df = pd.read_csv(file_path, encoding="utf-8-sig")


class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze()
        }

def convert_to_embeddings(df, column_names, max_length=512, batch_size=32, device='cuda' if torch.cuda.is_available() else 'cpu'):
  
    # Load model and tokenizer once
    print("Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained('asafaya/bert-base-arabic')
    model = AutoModel.from_pretrained('asafaya/bert-base-arabic')
    model = model.to(device)
    model.eval()
    
    for column_name in column_names:
        print(f"\nProcessing column: {column_name}")
        
        # Create dataset and dataloader
        texts = df[column_name].tolist()
        dataset = TextDataset(texts, tokenizer, max_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        
        # Process batches
        embeddings = []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Generating embeddings"):
                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                # Generate embeddings
                outputs = model(input_ids, attention_mask=attention_mask)
                
                # Compute mean pooling
                mask = attention_mask.unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
                masked_embeddings = outputs.last_hidden_state * mask
                summed = torch.sum(masked_embeddings, 1)
                counts = torch.clamp(mask.sum(1), min=1e-9)
                mean_pooled = summed / counts
                
                # Move to CPU and convert to numpy
                embeddings.append(mean_pooled.cpu().numpy())
        
        # Concatenate all batches
        all_embeddings = np.concatenate(embeddings, axis=0)
        
        # Store embeddings in DataFrame
        df[f"{column_name}_embedded"] = list(all_embeddings)
        
        print(f"Completed embedding generation for {column_name}")
    
    return df




In [16]:


def decode_categories(df):

    # Define the category mapping
    category_map = {
        "الأدب والخيال": "1",
        "الكتب الإسلامية": "10",
        "الاقتصاد والأعمال": "100",
        "الفلسفة": "1000",
        "الصحافة والإعلام": "10000",
        "الكتب السياسية": "100000",
        "العلوم والطبيعة": "1000000",
        "الأسرة والطفل": "10000000",
        "السير والمذكرات": "100000000",
        "الفنون": "1000000000",
        "التاريخ والجغرافيا": "10000000000",
        "الرياضة والتسلية": "100000000000",
        "الشرع والقانون": "1000000000000"
    }
    
    # Create reversed mapping
    reversed_category_map = {v: k for k, v in category_map.items()}
    
    # Convert category values to string to ensure proper matching
    df['Category'] = df['Category'].astype(str)
    
    # Function to safely map categories
    def safe_map_category(x):
        if pd.isna(x) or x == 'nan':
            return np.nan
        
        # Convert the input to a simple string of the number
        x_str = str(int(x))  # This removes leading zeros and converts to simple number string
        
        return reversed_category_map.get(x_str, x)
    
    # Apply the mapping
    df['Category_original'] = df['Category'].apply(safe_map_category)
    
    return df

print(df[['Category']].head(3))
df = decode_categories(df)
print(df[['Category']].head(3))
print(df[['Category_original']].head(3))

   Category
0     10000
1        10
2  10000000
   Category
0     10000
1        10
2  10000000
  Category_original
0  الصحافة والإعلام
1   الكتب الإسلامية
2     الأسرة والطفل


In [17]:
# Load the encoder later
with open("label_encoder_Subcategory.pkl", "rb") as f:
    label_encoder = pickle.load(f)
    
# Decode the 'Subcategory' column back to original values
df['Subcategory_original'] = label_encoder.inverse_transform(df['Subcategory'])

# Display a sample of the reversed data
print(df[['Subcategory', 'Subcategory_original']].head())


   Subcategory Subcategory_original
0           65   الندوات والمؤتمرات
1           54        القرآن وعلومه
2           71          شؤون المرأة
3           74           علم النبات
4           68          تاريخ الأدب


In [18]:
# we will decide the max_length based on the following results 
# Calculate the number of words in each text
df['word_count_Description'] = df['Description'].apply(lambda x: len(x.split()))
df['word_count_Title'] = df['Title'].apply(lambda x: len(x.split()))
# Analyze the distribution
print(df['word_count_Description'].describe())
print(df['word_count_Title'].describe())
df.drop(['word_count_Title', 'word_count_Description'], axis=1)

df = convert_to_embeddings(df, 
                         column_names=['Title'], 
                         max_length=20, 
                         batch_size=32)
#%75 of descriptions will be covered and 128 will avoid excessive padding for shorter descriptions
# and will truncates very long descriptions
df = convert_to_embeddings(df, 
                         column_names=["Description"], 
                         max_length=128, 
                         batch_size=32)

#Flatten embeddings into separate columns
df = pd.concat([df.drop(['Title', 'Description'], axis=1),
                df['Title'].apply(pd.Series),
                df['Description'].apply(pd.Series)], axis=1)


count    3299.000000
mean       94.642316
std        95.139226
min         2.000000
25%        56.000000
50%        63.000000
75%        72.000000
max      1374.000000
Name: word_count_Description, dtype: float64
count    3299.000000
mean        4.825402
std         2.597022
min         1.000000
25%         3.000000
50%         4.000000
75%         6.000000
max        20.000000
Name: word_count_Title, dtype: float64
Loading model and tokenizer...

Processing column: Title


Generating embeddings: 100%|██████████| 104/104 [00:18<00:00,  5.55it/s]


Completed embedding generation for Title
Loading model and tokenizer...

Processing column: Description


Generating embeddings: 100%|██████████| 104/104 [01:26<00:00,  1.21it/s]


Completed embedding generation for Description


In [19]:
# the end result of the dataset in the training we will use Subcategory_original, Category_original
# in catogory format after decoding to utilize catboost 
print (df.head(1))

   Author  Pages  Publication year  Publisher Category  Subcategory  Price  \
0    2073     40              2003        145    10000           65  16.88   

  Page Range Category_original Subcategory_original  word_count_Description  \
0       0-50  الصحافة والإعلام   الندوات والمؤتمرات                      71   

   word_count_Title                                     Title_embedded  \
0                 7  [0.64062077, -0.59859043, 0.09329636, -0.49279...   

                                Description_embedded  \
0  [0.4787133, -0.15091261, 0.24697617, -0.482517...   

                                                 0  \
0  التشبيك وميثاق الممارسة في عمل المنظمات الأهلية   

                                                   0  
0  تقرير يوثق أعمال ورشة عمل 1995 عن محاولة صياغة...  


In [21]:
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np

# Load the tokenizer and model for Arabic BERT
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
bert_model = AutoModel.from_pretrained("asafaya/bert-base-arabic").to("cuda" if torch.cuda.is_available() else "cpu")

# 1️. Prepare the dataset for SVM
X = df['Description_embedded'].apply(np.array).tolist()   # Convert embeddings column to lists
X = np.array(X)  # Convert the list to a numpy array

# The target variable (category labels)
y = df['Category_original']

# 2. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize the SVM model
svm_model = SVC(kernel="linear", probability=True, random_state=42)

# 4. Train the model
svm_model.fit(X_train, y_train)

# 4. **Perform Cross-Validation**
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f"Cross-validation scores: {cross_val_scores}")
print(f"Mean CV score: {cross_val_scores.mean()}")

# 5. Calculate Training Accuracy
y_train_pred = svm_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")

# 6. Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Save predictions to a CSV file
pd.DataFrame(y_pred, columns=['Predictions']).to_csv("y_pred_svm.csv", index=False)

# 7. Calculate Test Accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# 9. Classify a new description (example)
example_description = "طفل"  # Example new description
device = "cuda" if torch.cuda.is_available() else "cpu"

# Convert the new description into model-compatible input using the tokenizer
example_inputs = tokenizer(example_description, truncation=True, max_length=128, padding='max_length', return_tensors='pt').to(device)

with torch.no_grad():
    outputs = bert_model(example_inputs['input_ids'], attention_mask=example_inputs['attention_mask'])
    mask = example_inputs['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
    masked_embeddings = outputs.last_hidden_state * mask
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    mean_pooled = summed / counts  # final embedding

# Convert embedding to NumPy for SVM prediction
example_embedding = mean_pooled.cpu().numpy()

# 10. Predict the category using the trained SVM model
example_category = svm_model.predict(example_embedding)
print(f"Predicted Category: {example_category}")

Cross-validation scores: [0.56060606 0.59659091 0.59659091 0.61174242 0.57874763]
Mean CV score: 0.5888555862227589
Training Accuracy: 0.9913
Test Accuracy: 0.6212
Predicted Category: ['الأسرة والطفل']
